## summary

this kernel using the google-news-vector-300 to trian the bilstm network.

we try two dataset raw, pre-process(refer to the pre-fit-embedding.ipynb)

using raw dataset the Private Score is 0.93930 and Public Score is 0.93277

using pre-process dataset the Pricate Score is 0.96446 and the Public Score is 0.96443

the train method is bilstm network using keras(backend tensorflow) in kaggle kernel

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin.gz
/kaggle/input/processed/process_train.csv
/kaggle/input/processed/process_test.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv


In [2]:
from gensim.models import KeyedVectors
embedding_index = KeyedVectors.load_word2vec_format('../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [3]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
pre_train = pd.read_csv('../input/processed/process_train.csv')
pre_train_comment = pre_train['comment_text'].fillna('')
pre_test = pd.read_csv('../input/processed/process_test.csv')
pre_test_comment = pre_test['comment_text'].fillna('')

In [5]:
y = pre_train[labels].values

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import gc

Using TensorFlow backend.


In [7]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(pre_train_comment))
pre_train_comment = tokenizer.texts_to_sequences(pre_train_comment)
pre_test_comment = tokenizer.texts_to_sequences(pre_test_comment)

In [8]:
maxlen = 200
X_t = pad_sequences(pre_train_comment, maxlen=maxlen)
X_te = pad_sequences(pre_test_comment, maxlen=maxlen)

In [9]:
def loadEmbeddingMatrix(pre_vector):
    embed_size = 300
    embeddings_index = dict()
    for word in pre_vector.wv.vocab:
        embeddings_index[word] = pre_vector.word_vec(word)
    print('Loaded %s word vectors.' % len(embeddings_index))
    
    gc.collect()
    all_embs = np.stack(list(embeddings_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    
    nb_words = len(tokenizer.word_index)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    gc.collect()

    embeddedCount = 0
    for word, i in tokenizer.word_index.items():
        i -= 1
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            embeddedCount += 1
    print('total embedded:', embeddedCount,'common words')
    
    del(embeddings_index)
    gc.collect()

    return embedding_matrix
    

In [11]:
embedding_matrix.shape

(211620, 300)

In [12]:
inp = Input(shape=(maxlen, ))
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)
x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(6, activation='sigmoid')(x)

In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint = ModelCheckpoint('../input/weights_word2vec.best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_loss", mode="max", patience=20)
callbacks_list = [checkpoint, early]

In [14]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          63486000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 120)          173280    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 120)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6050      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0   

In [16]:
batch_size = 128
epochs = 5
result = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 725s 5ms/step - loss: 0.1054 - accuracy: 0.9694 - val_loss: 0.0693 - val_accuracy: 0.9775

Epoch 00001: val_loss improved from -inf to 0.06927, saving model to ../input/weights_word2vec.best.hdf5
Epoch 2/5
143613/143613 [==============================] - 706s 5ms/step - loss: 0.0661 - accuracy: 0.9784 - val_loss: 0.0599 - val_accuracy: 0.9798

Epoch 00002: val_loss did not improve from 0.06927
Epoch 3/5
143613/143613 [==============================] - 702s 5ms/step - loss: 0.0583 - accuracy: 0.9802 - val_loss: 0.0570 - val_accuracy: 0.9804

Epoch 00003: val_loss did not improve from 0.06927
Epoch 4/5
143613/143613 [==============================] - 704s 5ms/step - loss: 0.0544 - accuracy: 0.9812 - val_loss: 0.0548 - val_accuracy: 0.9810

Epoch 00004: val_loss did not improve from 0.06927
Epoch 5/5
143613/143613 [==============================] - 697s 5ms/step - loss: 0.0516 - a

In [18]:
y_test = model.predict(X_te)

In [19]:
sample_submission = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv")
sample_submission[labels] = y_test

sample_submission.to_csv("submission.csv")